<a href="https://colab.research.google.com/github/OmarElsebaey001/trial00/blob/master/clean_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**How to run**


1.   Upload helper.py to a folder on the drive
      *   For example create a folder called "trading app" and then upload the helper.py to this folder


2. Set the path to the helper.py in the next cell in this line 
      * sys.path.append('/content/drive/MyDrive/trading_app')

3. For single run 
      * Set the variables as elaborated in the cell#2

4. For multi runs with a file containing the input files 
      * Call the funciton generate_net_threshold_for_multi_runs and pass the file to it like Cell#3
      * The columns of the file should have columns names like this : 
        * input_file,output_file,ticker,start_date,end_date


In [1]:
import os
from google.colab import drive
import sys
import pandas as pd
import numpy as np 
pd.options.mode.chained_assignment = None
from datetime import  date,time
import os
from datetime import datetime 
import logging
import numpy as np 
from datetime import datetime, timedelta,time,date
import pandas as pd 

In [2]:
def add_n_minutes(t1,n) :
    t1 = datetime.combine(datetime.strptime("00:00", '%H:%M').date(),t1)
    time_zero = datetime.strptime('00:00', '%H:%M')
    n_minutes = datetime.strptime('00:0{}'.format(n), '%H:%M')
    return (t1-time_zero+n_minutes).time()
def create_ideal_times(start_time,end_time,minute_step):
    start_time = datetime.strptime(start_time, '%H:%M')
    end_time   = datetime.strptime(end_time, '%H:%M')
    step       = datetime.strptime(minute_step, '%H:%M')
    time_zero = datetime.strptime('00:00', '%H:%M')
    golden_times = [] 
    current_time = start_time 
    while (current_time <= end_time) :
        golden_times.append(current_time)
        current_time = (current_time  - time_zero + step )
    golden_times = [i.time() for i in golden_times ]
    return golden_times
def select_last_close(d,atime,plus_two) : 
    d.insert(5, 'New_Close', "SHOULD_DROP" )
    avalue = list(d[d['Time'] == plus_two]['Close'])[0]
    d.loc[d['Time'] == atime, 'New_Close'] = avalue
    return d
def get_valid_chuncks(df,ideal_list,step) :
    discrepency = 0 
    all_dates_df = pd.DataFrame()
    for adate in (set(df.Date)):
        date_df = df.loc[df['Date'] == adate]
        valid_date_batch = pd.DataFrame()
        for atime in ideal_list :
            valid_time_batch = pd.DataFrame()
            plus_one = add_n_minutes(atime,1)
            plus_two = add_n_minutes(plus_one,1)
            valid_time_batch = date_df.loc[date_df['Time'].isin([atime,plus_one,plus_two])]
            if (valid_time_batch.shape[0] == 3 ):
                valid_time_batch = select_last_close(valid_time_batch,atime,plus_two)
                valid_date_batch = valid_date_batch.append(valid_time_batch)
            elif (valid_time_batch.shape[0] in [1,2] ):
                discrepency += 1 
                print("[WARN] Found discrepency at Date {}".format(adate))
                print("           At Times : {} {} {} ".format(atime,plus_one,plus_two))
        total_date_batch = valid_date_batch.shape[0]
        all_dates_df = all_dates_df.append(valid_date_batch)
    all_dates_df.sort_values(by=['Date/Time'],ignore_index=True,inplace=True)
    return all_dates_df,discrepency   

def drop_old_close(df):
    df.drop(['Close'], axis = 1,inplace=True)
    df.rename(columns = {'New_Close':'Close'}, inplace = True)
    df = df[df['Close'] != "SHOULD_DROP" ]
    df['Close'] = df['Close'].astype(float)
    df.reset_index(inplace=True,drop=True)
    return df 
def separate_date_time(df) : 
    df['Date/Time'] = pd.to_datetime(df['Date/Time'], dayfirst=True )
    df['Date']      = df['Date/Time'].dt.date
    df['Time']      = df['Date/Time'].dt.time
    return df 
def get_swing_value_from_ticker (ticker_value) : 
    adict = {"BANKNIFTY":300,"NIFTY":100}
    if (ticker_value == None) : 
      return 300 
    return adict[ticker_value]
def select_date_time_range(df,start_dt,end_dt):
    res = df[(df['Date/Time'] >= start_dt) & (df['Date/Time'] <= end_dt) ]
    res.sort_values(by=['Date/Time'],inplace=True,ignore_index=True)
    return res
def round_up(df) : 
    df['ROUNDUP'] = np.round(df["Close"]/100)*100
    df['ROUNDUP'] = df['ROUNDUP'].astype(int)
    return df 
def generate_pe(sent_df,swing) :
    df = sent_df.copy()
    df['Strike'] = df['ROUNDUP'] + swing
    df['Type'] = 'PE'
    df['Contract'] = df['Strike'].astype(str) + df['Type']
    return df 
def generate_ce(sent_df,swing) :
    df = sent_df.copy() 
    df['Strike'] = df['ROUNDUP'] - swing
    df['Type'] = 'CE'
    df['Contract'] = df['Strike'].astype(str) + df['Type']
    return df 
def update_close_value(main_df) : 
    main_df["New_Close"] = 0.0
    for i in range(len(main_df)):
        advance = i+2 
        if (advance < len(main_df)):
            if (main_df['Date'][i] == main_df['Date'][advance]):
                advance_value = main_df['Close'][advance]
            else:
                advance_value = main_df['Close'][i]
        else : 
            advance_value = main_df['Close'][i]
        main_df['New_Close'][i] = advance_value
def delay_3_min (df) : 
    df['Date/Time']  = df['Date/Time'] + np.timedelta64(3, 'm')
    df['Time'] = df['Date/Time'].dt.time
    df['Date'] = df['Date/Time'].dt.date
    return df
def select_ticker_section(df,ticker) : 
    pass  
MARKET_START = "09:15"
MARKET_END   = "15:21"
MARKET_STEP  = "00:03"
ideal_list   = create_ideal_times(MARKET_START,MARKET_END,MARKET_STEP)


In [3]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
def generate_net_threshold_for_multi_runs(afile) : 
  df = pd.read_csv(afile)
  df.fillna("", inplace=True)
  for i in range(len(df)) : 
    outputfile         = df['output_file'][i]
    input_file         = df['input_file'][i]
    START_DATE         = df['start_date'][i]
    END_DATE           = df['end_date'][i]
    TICKER             = str(df['ticker'][i])
    print("[INFO] Generating for file number: ",i,)
    generate_net_threshold(input_file,START_DATE,END_DATE,TICKER,outputfile)
  print("[INFO] Finished generating net thresholds for all files")
def select_ticker_df(df,ticker) : 
  selected = df 
  if(ticker in ['NIFTY','BANKNIFTY']):
    selected = df[df['Ticker'] == ticker]
  return selected

def generate_1_min_and_pe_and_ce(file_name,ticker,START_DATE,END_DATE):
  swing_value                   = get_swing_value_from_ticker(ticker)
  main_raw                      = pd.read_csv(file_name)
  main_ticker_selected          = select_ticker_df(main_raw,ticker)
  print("[INFO] Selected {} entries with Ticker specified".format(main_ticker_selected.shape[0]))
  main_date_time                = separate_date_time(main_ticker_selected)
  main_dt_selected              = select_date_time_range(main_date_time,START_DATE,END_DATE)
  print("[INFO] Filtered entries out of Date selection " )
  main_df_1_min                 = main_dt_selected.copy()
  print("[INFO] 1 Min frame created with {} entries".format(main_df_1_min.shape[0]))
  if(main_df_1_min.shape[0] == 0 ) : 
    return pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
  validated_df_discr            = get_valid_chuncks(main_dt_selected,ideal_list,MARKET_STEP)
  main_df_sel_val_new_close     = validated_df_discr[0]
  discrepency                   = validated_df_discr[1]
  main_df_closed                = drop_old_close(main_df_sel_val_new_close)
  main_df_rounded               = round_up(main_df_closed)
  main_df_delayed               = delay_3_min(main_df_rounded)
  main_3_min_pe                 = generate_pe(main_df_delayed,swing_value)
  main_3_min_ce                 = generate_ce(main_df_delayed,swing_value)
  print("[INFO] 3 Min PE Contract created with {} entries".format(main_3_min_pe.shape[0]))
  print("[INFO] 3 Min CE Contract created with {} entries".format(main_3_min_ce.shape[0]))
  return main_df_1_min,main_3_min_pe,main_3_min_ce

def generate_threshold_files(main_df,px_df):
    full_df = main_df
    df2 = px_df
    arr = np.array([])
    check_times = [3,6,10,15,20,24,30,40,50,60,75,96,120]
    lim_time = time(15,28)
    df_cur = df2.loc[df2['Date'] == df2['Date'][0]]
    terminal_time = lim_time
    df2 = df2.assign(End_Time=pd.to_datetime(df2['Date/Time']).dt.time)
    df2 = df2.assign(Return=pd.to_numeric([0]*len(df2)))
    for threshold_min in range(1,50):
        hold_order = 0
        cur_max_ret = 0
        ct = 0
        time_ind = -1
        df = full_df.loc[full_df['Date'] == full_df['Date'][0]]
        df.reset_index(drop=True, inplace=True)
        df_cur = df2.loc[df2['Date'] == full_df['Date'][0]]
        df_cur.reset_index(drop=True, inplace=True)
        for j in range(len(df_cur)):
            id_val_temp = (df.index[df['Time'] == df_cur['Time'][j]])
            id_val = 0
            try:
                id_val = id_val_temp[0]
            except:
                continue
            i = id_val
            opt_type_temp = df_cur['Contract'][j][-2:]
            cur_max_ret = 0

            if(opt_type_temp=='CE'):
                temp_val = df['Close'][i] - df['Open'][i]
                if(temp_val>cur_max_ret):
                    cur_max_ret = temp_val
                    prev_order = j
                    time_ind = i
                    hold_order = 1
                    opt_type = opt_type_temp
                else:
                    if(cur_max_ret-temp_val>threshold_min):
                        df_cur['End_Time'][j] = df_cur['Time'][j]
                        df_cur['Return'][j] = temp_val
                        prev_order = -1
                        hold_order = 0
                        continue

                    else:
                        prev_order = j
                        time_ind = i
                        hold_order = 1
                        opt_type = opt_type_temp
            else:
                temp_val = df['Open'][i] - df['Close'][i]
                if(temp_val>cur_max_ret):
                    cur_max_ret = temp_val
                    prev_order = j
                    time_ind = i
                    hold_order = 1
                    opt_type = opt_type_temp
                else:

                    if(cur_max_ret-temp_val>threshold_min):

                        df_cur['End_Time'][j] = df_cur['Time'][j]
                        df_cur['Return'][j] = temp_val
                        prev_order = -1
                        hold_order = 0
                        continue
                    else:
                        prev_order = j
                        time_ind = i
                        hold_order = 1
                        opt_type = opt_type_temp
            for i in range(id_val+1,len(df)):
                    if(df['Time'][i]==terminal_time or i==len(df)-1):

                            check_val = df_cur.index[df_cur['Time'] == df['Time'][i]]
                            ct +=1
                            df_cur['End_Time'][prev_order] = df['Time'][i]
                            if(opt_type=='CE'):
                                df_cur['Return'][prev_order] = df['Close'][i]-df['Open'][time_ind]
                            else:
                                df_cur['Return'][prev_order] = df['Open'][time_ind]-df['Close'][i]
                            prev_order = -1
                            hold_order = 0
                            break
                    if(opt_type=='CE'):
                        temp_val = df['Close'][i] - df['Open'][time_ind]
                        if(temp_val>cur_max_ret):
                            cur_max_ret = temp_val
                        else:
                            if(cur_max_ret-temp_val>threshold_min):
                                ct +=1
                                df_cur['End_Time'][prev_order] = df['Time'][i]
                                df_cur['Return'][prev_order] = temp_val
                                prev_order = -1
                                hold_order = 0
                                break

                    else:
                        temp_val = df['Open'][time_ind] - df['Close'][i]
                        if(temp_val>cur_max_ret):
                            cur_max_ret = temp_val
                        else:
                            if(cur_max_ret-temp_val>threshold_min):
                                ct +=1
                                df_cur['End_Time'][prev_order] = df['Time'][i]
                                df_cur['Return'][prev_order] = temp_val
                                prev_order = -1
                                hold_order = 0
                                break    
        res = df_cur
        for d in full_df['Date'].unique()[1:]:
            hold_order = 0
            cur_max_ret = 0
            ct = 0
            time_ind = -1
            df = full_df.loc[full_df['Date'] == d]
            df.reset_index(drop=True, inplace=True)
            df_cur = df2.loc[df2['Date'] == d]
            df_cur.reset_index(drop=True, inplace=True)
            for j in range(len(df_cur)):
                id_val_temp = (df.index[df['Time'] == df_cur['Time'][j]])
                id_val = 0
                try:
                    id_val = id_val_temp[0]
                except:
                    continue
                i = id_val
                opt_type_temp = df_cur['Contract'][j][-2:]
                cur_max_ret = 0
                if(opt_type_temp=='CE'):
                    temp_val = df['Close'][i] - df['Open'][i]
                    if(temp_val>cur_max_ret):
                        cur_max_ret = temp_val
                        prev_order = j
                        time_ind = i
                        hold_order = 1
                        opt_type = opt_type_temp
                    else:
                        if(cur_max_ret-temp_val>threshold_min):
                            df_cur['End_Time'][j] = df['Time'][i]
                            df_cur['Return'][j] = temp_val
                            prev_order = -1
                            hold_order = 0
                            continue

                        else:
                            prev_order = j
                            time_ind = i
                            hold_order = 1
                            opt_type = opt_type_temp
                else:
                    temp_val = df['Open'][i] - df['Close'][i]
                    if(temp_val>cur_max_ret):
                        cur_max_ret = temp_val
                        prev_order = j
                        time_ind = i
                        hold_order = 1
                        opt_type = opt_type_temp
                    else:
                        if(cur_max_ret-temp_val>threshold_min):
                            df_cur['End_Time'][j] = df['Time'][i]
                            df_cur['Return'][j] = temp_val
                            prev_order = -1
                            hold_order = 0
                            continue

                        else:
                            prev_order = j
                            time_ind = i
                            hold_order = 1
                            opt_type = opt_type_temp
                for i in range(id_val+1,len(df)):
                        if(df['Time'][i]==terminal_time or i==len(df)-1):
                                check_val = df_cur.index[df_cur['Time'] == df['Time'][i]]
                                ct +=1
                                df_cur['End_Time'][prev_order] = df['Time'][i]
                                if(opt_type=='CE'):
                                    df_cur['Return'][prev_order] = df['Close'][i]-df['Open'][time_ind]
                                else:
                                    df_cur['Return'][prev_order] = df['Open'][time_ind]-df['Close'][i]
                                prev_order = -1
                                hold_order = 0
                                break
                        
                        if(opt_type=='CE'):
                            temp_val = df['Close'][i] - df['Open'][time_ind]
                            if(temp_val>cur_max_ret):
                                cur_max_ret = temp_val
                            else:
                                if(cur_max_ret-temp_val>threshold_min):
                                    ct +=1
                                    df_cur['End_Time'][prev_order] = df['Time'][i]
                                    df_cur['Return'][prev_order] = temp_val
                                    prev_order = -1
                                    hold_order = 0
                                    break
                        else:
                            temp_val = df['Open'][time_ind] - df['Close'][i]
                            if(temp_val>cur_max_ret):
                                cur_max_ret = temp_val
                            else:
                                if(cur_max_ret-temp_val>threshold_min):
                                    ct +=1
                                    df_cur['End_Time'][prev_order] = df['Time'][i]
                                    df_cur['Return'][prev_order] = temp_val
                                    prev_order = -1
                                    hold_order = 0
                                    break    

            res = res.append(df_cur,ignore_index=True)
        arr = np.append(arr,np.sum(res['Return'].values))
        res.to_csv('Total_Result_Threshhold_{}_limit.csv'.format(threshold_min),index=False)
    res.to_csv('Total_Result_Threshhold_all.csv',index=False)

def get_max_threshold_frame(full_df,arr) : 
    tp = []
    for i in range(1,50):
        tp.append(i)
    summary_df = pd.DataFrame(columns=tp,index=full_df['Date'].unique())
    for i in range(1,50):
        res = pd.read_csv('Total_Result_Threshhold_{}_limit.csv'.format(i))
        res['Date'] = (pd.to_datetime(res['Date']).dt.date)
        for d in full_df['Date'].unique(): 
            df_cur_temp = res.loc[res['Date'] == d]
            summary_df[i][d] = np.sum(df_cur_temp['Return'].values)
    max_threshold = np.where(arr == np.amax(arr))[0][0] + 1
    max_thr_df = pd.read_csv("Total_Result_Threshhold_{}_limit.csv".format(max_threshold))
    max_thr_df = max_thr_df[max_thr_df['Return'] > 19]
    max_thr_df.reset_index(inplace=True,drop=True)
    max_thr_df['Date/Time'] = pd.to_datetime(max_thr_df['Date/Time'], dayfirst=True )
    max_thr_df['Date'] = max_thr_df['Date/Time'].dt.date
    max_thr_df['Time'] = max_thr_df['Date/Time'].dt.time
    return max_thr_df

def generate_net_threshold(input_file,START_DATE,END_DATE,TICKER,output_file):
  ticker_dict = {"BANKNIFTY" : "Swing will be +/- 300","NIFTY" : "Swing will be +/- 100"} 
  input_file  = input_file
  output_file = output_file
  print("[INFO] Processing input file: {}".format(input_file))
  ticker = TICKER
  if (ticker == "" ) : 
    ticker = None
  if(ticker != None) : 
    print("[INFO] Ticker is set to : {} and {}".format(ticker,ticker_dict[ticker]))
  else:
    print("[INFO] No Ticker specified")
  main_df,pe_df,ce_df = generate_1_min_and_pe_and_ce(input_file,ticker,START_DATE,END_DATE)
  if (main_df.shape[0] == 0 ) : 
    print("[INFO] Didn't Find any data matching the selection criteria")
    return
  print("[INFO] New contracts length: PE={} CE={}".format(pe_df.shape[0],ce_df.shape[0]))
  pe_thr_arr    = generate_threshold_files(main_df,pe_df)
  max_thr_pe_df = get_max_threshold_frame(main_df,pe_thr_arr)
  print("[INFO] Generated Threshold files for PE and selected the max")
  ce_thr_arr    = generate_threshold_files(main_df,ce_df)
  max_thr_ce_df = get_max_threshold_frame(main_df,ce_thr_arr)
  print("[INFO] Generated Threshold files for CE and selected the max")
  pe_plus_ce    = max_thr_pe_df.append(max_thr_ce_df)
  print("[INFO] Appended PE&CE Max Thresholds")

  pe_plus_ce["Date/Time"] = pd.to_datetime(pe_plus_ce['Date/Time'], dayfirst=True )
  pe_plus_ce["Time"]      = pd.to_datetime(pe_plus_ce['Date/Time'], dayfirst=True ).dt.time
  pe_plus_ce["Date"]      = pd.to_datetime(pe_plus_ce['Date/Time'], dayfirst=True ).dt.date
  
  pe_plus_ce.sort_values(by=['Date/Time'], inplace=True)
  pe_plus_ce.reset_index(inplace=True,drop=True)
  arr = generate_threshold_files(main_df,pe_plus_ce)
  print("[INFO] Generated threshold files for appended PE&CE Contracts")
  net_max_thr = get_max_threshold_frame(main_df,arr)
  print("[INFO] Selected Max threshold for PE&CE")
  print("[INFO] Writing the output to: {} ".format(output_file))
  net_max_thr.to_csv(output_file,index=False)

Single Run example

In [5]:
outputfile         = '/content/drive/MyDrive/trading_app/net_new_bug.csv'
input_file         = '/content/drive/MyDrive/trading_app/noticker.csv'
START_DATE         = '17-02-22 15:21'
END_DATE           = '18-02-22 15:27'
TICKER             = ""
generate_net_threshold(input_file,START_DATE,END_DATE,TICKER,outputfile)

[INFO] Processing input file: /content/drive/MyDrive/trading_app/noticker.csv
[INFO] No Ticker specified
[INFO] Selected 747 entries with Ticker specified
[INFO] Filtered entries out of Date selection 
[INFO] 1 Min frame created with 378 entries
[WARN] Found discrepency at Date 2022-02-18
           At Times : 09:48:00 09:49:00 09:50:00 
[INFO] 3 Min PE Contract created with 122 entries
[INFO] 3 Min CE Contract created with 122 entries
[INFO] New contracts length: PE=122 CE=122
[INFO] Generated Threshold files for PE and selected the max
[INFO] Generated Threshold files for CE and selected the max
[INFO] Appended PE&CE Max Thresholds
[INFO] Generated threshold files for appended PE&CE Contracts
[INFO] Selected Max threshold for PE&CE
[INFO] Writing the output to: /content/drive/MyDrive/trading_app/net_new_bug.csv 


Multiple Runs from a csv file 

In [6]:
multi_files_input = '/content/drive/MyDrive/trading_app/input.csv'
generate_net_threshold_for_multi_runs(multi_files_input)

[INFO] Generating for file number:  0
[INFO] Processing input file: /content/drive/MyDrive/trading_app/banknifty.csv
[INFO] Ticker is set to : BANKNIFTY and Swing will be +/- 300
[INFO] Selected 747 entries with Ticker specified
[INFO] Filtered entries out of Date selection 
[INFO] 1 Min frame created with 378 entries
[WARN] Found discrepency at Date 2022-02-18
           At Times : 09:48:00 09:49:00 09:50:00 
[INFO] 3 Min PE Contract created with 122 entries
[INFO] 3 Min CE Contract created with 122 entries
[INFO] New contracts length: PE=122 CE=122
[INFO] Generated Threshold files for PE and selected the max
[INFO] Generated Threshold files for CE and selected the max
[INFO] Appended PE&CE Max Thresholds
[INFO] Generated threshold files for appended PE&CE Contracts
[INFO] Selected Max threshold for PE&CE
[INFO] Writing the output to: NEW1.csv 
[INFO] Generating for file number:  1
[INFO] Processing input file: /content/drive/MyDrive/trading_app/nifty.csv
[INFO] Ticker is set to : NIF